In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from brown_clustering import Corpus, BrownClustering

# Base Run

In [2]:
train_data = pd.read_csv("./data/emotion-labels-train.csv")
test_data = pd.read_csv("./data/emotion-labels-test.csv")

In [3]:
algos = [RandomForestClassifier(), LogisticRegression()]

In [4]:
count_vectorizer=CountVectorizer(analyzer='word', ngram_range=(1, 2))
x = count_vectorizer.fit_transform(train_data["text"])
print(x.shape)

(3613, 43357)


In [5]:
def calculate_accuracy(predicted_data):
	count_equals=0
	for idx, row in test_data.iterrows():
		if row["label"]==predicted_data[idx]:
			count_equals+=1
	print(f"accuracy={count_equals/len(predicted_data)}")

In [6]:

for algo_under_test in algos:
	pipe = Pipeline([('vectorizer', count_vectorizer),('algo', algo_under_test)])
	pipe.fit(train_data["text"], train_data["label"])
	predicted = pipe.predict(test_data["text"])
	calculate_accuracy(predicted)

accuracy=0.7991725015913431
accuracy=0.7721196690006366


# Data Processing

In [11]:
train_as_list = train_data["text"].tolist()[:100]
# toeknize
tokenizer = RegexpTokenizer(r'\w+')
train_data_tokenized = [w.lower() for w in train_as_list]
train_data_tokenized = [tokenizer.tokenize(i) for i in train_data_tokenized]

In [15]:
corpus = Corpus(train_data_tokenized, 0.001)
clustering = BrownClustering(corpus, 10)
clustering.train()

IN TRAIN
1	2021-08-02 20:02:30.967140


NotImplementedError: 